<a href="https://colab.research.google.com/github/CodeofRahul/YouTube-Screenshot-Notes-Generator/blob/main/YouTube_to_PDF_conveter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# YouTube Video → Screenshot-based PDF Notes Generator (ROBUST)
# Single Cell | Error Handling | User Feedback | Google Colab
# ============================================================

# ----------------------------
# 1. Install required libraries
# ----------------------------
try:
    !pip install -q yt-dlp opencv-python pillow reportlab
except Exception as e:
    print("❌ Failed to install dependencies.")
    raise e

# ----------------------------
# 2. Import libraries
# ----------------------------
import os
import cv2
from PIL import Image
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from yt_dlp import YoutubeDL
from google.colab import files

# ----------------------------
# 3. Ask user for YouTube URL
# ----------------------------
VIDEO_URL = input("🔗 Enter YouTube video URL: ").strip()

if not VIDEO_URL:
    print("❌ Error: YouTube URL cannot be empty.")
    raise SystemExit

# ----------------------------
# 4. User configuration
# ----------------------------
SCREENSHOT_INTERVAL_SEC = 30     # Screenshot every N seconds
OUTPUT_DIR = "yt_notes"
PDF_NAME = "youtube_notes.pdf"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------------
# 5. Download YouTube video
# ----------------------------
def download_video(url, output_path):
    """
    Downloads a YouTube video using yt-dlp
    """
    try:
        ydl_opts = {
            'outtmpl': output_path,
            'format': 'mp4',
            'quiet': True
        }
        with YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            return ydl.prepare_filename(info)
    except Exception as e:
        print("❌ Failed to download the video.")
        print("👉 Possible reasons:")
        print("   - Invalid YouTube URL")
        print("   - Private / age-restricted video")
        print("   - Network issue")
        raise e

print("⬇️ Downloading video...")
video_path = download_video(VIDEO_URL, f"{OUTPUT_DIR}/video.mp4")
print("✅ Video downloaded successfully")

# ----------------------------
# 6. FAST Screenshot Extraction
# ----------------------------
def extract_screenshots_fast(video_path, interval_sec, output_dir):
    """
    Extract screenshots by jumping directly to timestamps
    (much faster than frame-by-frame processing)
    """
    if not os.path.exists(video_path):
        print("❌ Video file not found.")
        raise SystemExit

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("❌ Failed to open video file.")
        raise SystemExit

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if fps == 0 or total_frames == 0:
        print("❌ Unable to read video metadata.")
        cap.release()
        raise SystemExit

    duration_sec = int(total_frames / fps)
    screenshot_count = 0

    print(f"🎥 Video duration: {duration_sec} seconds")
    print("📸 Extracting screenshots...")

    for t in range(0, duration_sec, interval_sec):
        cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
        ret, frame = cap.read()

        if not ret:
            print(f"⚠️ Skipped frame at {t} sec (could not read frame)")
            continue

        # Resize to reduce processing time & PDF size
        frame = cv2.resize(frame, (1280, 720))

        screenshot_path = os.path.join(
            output_dir, f"screenshot_{screenshot_count:03d}.jpg"
        )
        cv2.imwrite(screenshot_path, frame)
        screenshot_count += 1

    cap.release()

    if screenshot_count == 0:
        print("❌ No screenshots were extracted.")
        raise SystemExit

    print(f"⚡ Extracted {screenshot_count} screenshots successfully")

extract_screenshots_fast(video_path, SCREENSHOT_INTERVAL_SEC, OUTPUT_DIR)

# ----------------------------
# 7. Create PDF from screenshots
# ----------------------------
def create_pdf_from_images(image_folder, pdf_path):
    """
    Convert screenshots into a single PDF
    """
    images = sorted([
        os.path.join(image_folder, img)
        for img in os.listdir(image_folder)
        if img.endswith(".jpg")
    ])

    if not images:
        print("❌ No images found to create PDF.")
        raise SystemExit

    try:
        c = canvas.Canvas(pdf_path, pagesize=A4)
        page_width, page_height = A4

        for img_path in images:
            img = Image.open(img_path)
            img_width, img_height = img.size

            scale = min(page_width / img_width, page_height / img_height)
            new_width = img_width * scale
            new_height = img_height * scale

            x = (page_width - new_width) / 2
            y = (page_height - new_height) / 2

            c.drawImage(img_path, x, y, new_width, new_height)
            c.showPage()

        c.save()
        print(f"📄 PDF created successfully: {pdf_path}")

    except Exception as e:
        print("❌ Failed to create PDF.")
        raise e

pdf_path = os.path.join(OUTPUT_DIR, PDF_NAME)
create_pdf_from_images(OUTPUT_DIR, pdf_path)

# ----------------------------
# 8. Download PDF
# ----------------------------
try:
    files.download(pdf_path)
    print("🎉 Notes generation completed successfully!")
except Exception:
    print("⚠️ PDF created, but auto-download failed.")
    print(f"👉 You can manually download it from: {pdf_path}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.5 MB/s eta 0:00:00
🔗 Enter YouTube video URL: https://youtu.be/HVvN4T6AzSk?si=jRn3efAjb-NvWrUN
⬇️ Downloading video...


✅ Video downloaded successfully
🎥 Video duration: 3928 seconds
📸 Extracting screenshots...
⚡ Extracted 131 screenshots successfully
📄 PDF created successfully: yt_notes/youtube_notes.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Notes generation completed successfully!
